In [1]:
from theano.sandbox import cuda
cuda.use('gpu2')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(2), GPU number 0 is already in use.


In [39]:
%matplotlib inline
import sys
sys.path.append('../')  
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

In [4]:
#path = "data/ml-20m/"
path = "../data/ml-latest-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

In [5]:
import pandas as pd

## Set up data

We are trying to answer:
Who is going to like how what and how much?

Coll. filtering:
Lets find other people like you and find out what they thought about other things
People that rated movies you liked in similar ways
Asking people of their behavior is crap compared to looking at people's behavior.
MovieLens: http://grouplens.org/datasets/movielens/

In [6]:
#Starting using pandas.  Great way of dealing with structured data. 
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
len(ratings)

100004

In [8]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [9]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [10]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [11]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [12]:
movies.max()

163949

In [13]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [14]:
#These are the lenient factors, number of factors that 'describe' a movie or a user
n_factors = 50

In [15]:
np.random.seed = 42

In [16]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

## Create subset for Excel

In [17]:
#Group users by ratings
g=ratings.groupby('userId')['rating'].count()

In [19]:
g

userId
0        20
1        76
2        51
3       204
4       100
5        44
6        88
7       116
8        45
9        46
10       38
11       61
12       53
13       20
14     1700
15       29
16      363
17       51
18      423
19       98
20      162
21      220
22      726
23       21
24       26
25      172
26       23
27       50
28       22
29     1011
       ... 
641      36
642      24
643      39
644      30
645     169
646     150
647     256
648      90
649      29
650      20
651     267
652      51
653     626
654     105
655     128
656      20
657      60
658     142
659      92
660      33
661      58
662      26
663     519
664     434
665      40
666      68
667      20
668      37
669      31
670     115
Name: rating, dtype: int64

In [20]:
topUsers=g.sort_values(ascending=False)[:15]

In [21]:
topUsers

userId
546    2391
563    1868
623    1735
14     1700
72     1610
451    1340
467    1291
379    1063
310    1019
29     1011
293     947
508     923
579     922
212     910
211     876
Name: rating, dtype: int64

In [22]:
g=ratings.groupby('movieId')['rating'].count()

In [23]:
topMovies=g.sort_values(ascending=False)[:15]

In [24]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [25]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [26]:
#Creating a crosstab for ratings that top users have done for top movies.
#Why are top users 
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,27,49,57,72,79,89,92,99,143,179,180,197,402,417,505
userId,,,,,,,,,,,,,,,
14,3.0,5.0,1.0,3.0,4.0,4.0,5.0,2.0,5.0,5.0,4.0,5.0,5.0,2.0,5.0
29,5.0,5.0,5.0,4.0,5.0,4.0,4.0,5.0,4.0,4.0,5.0,5.0,3.0,4.0,5.0
72,4.0,5.0,5.0,4.0,5.0,3.0,4.5,5.0,4.5,5.0,5.0,5.0,4.5,5.0,4.0
211,5.0,4.0,4.0,3.0,5.0,3.0,4.0,4.5,4.0,NaN,3.0,3.0,5.0,3.0,NaN
212,2.5,NaN,2.0,5.0,NaN,4.0,2.5,NaN,5.0,5.0,3.0,3.0,4.0,3.0,2.0
293,3.0,NaN,4.0,4.0,4.0,3.0,NaN,3.0,4.0,4.0,4.5,4.0,4.5,4.0,NaN
310,3.0,3.0,5.0,4.5,5.0,4.5,2.0,4.5,4.0,3.0,4.5,4.5,4.0,3.0,4.0
379,5.0,5.0,5.0,4.0,NaN,4.0,5.0,4.0,4.0,4.0,NaN,3.0,5.0,4.0,4.0
451,4.0,5.0,4.0,5.0,4.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,2.0,3.5,5.0


## Dot product

In [28]:
from keras.layers.embeddings import Embedding
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-5))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-5))(movie_in)

In [42]:
from keras.layers import merge
from keras.models import Model
#A Merge layer can be used to merge a list of tensors into a single tensor, following some merge mode.
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [43]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80158 samples, validate on 19846 samples
Epoch 1/6
80158/80158 [==============================] - 6s - loss: 9.7687 - val_loss: 3.6671
Epoch 2/6
80158/80158 [==============================] - 6s - loss: 2.3168 - val_loss: 1.7974
Epoch 3/6
80158/80158 [==============================] - 6s - loss: 1.3650 - val_loss: 1.4443
Epoch 4/6
80158/80158 [==============================] - 6s - loss: 1.0856 - val_loss: 1.3275
Epoch 5/6
80158/80158 [==============================] - 6s - loss: 0.9543 - val_loss: 1.2775
Epoch 6/6
80158/80158 [==============================] - 6s - loss: 0.8752 - val_loss: 1.2579


##  Bias

In [44]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [45]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-5)

In [46]:
def create_bias(inp, n_in):
    return Flatten()(Embedding(n_in, 1, input_length=1)(inp))

In [47]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [48]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [49]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80158 samples, validate on 19846 samples
Epoch 1/6
80158/80158 [==============================] - 6s - loss: 8.6862 - val_loss: 2.9295
Epoch 2/6
80158/80158 [==============================] - 6s - loss: 1.8822 - val_loss: 1.4699
Epoch 3/6
80158/80158 [==============================] - 6s - loss: 1.1681 - val_loss: 1.2070
Epoch 4/6
80158/80158 [==============================] - 6s - loss: 0.9752 - val_loss: 1.1126
Epoch 5/6
80158/80158 [==============================] - 6s - loss: 0.8833 - val_loss: 1.0776
Epoch 6/6
80158/80158 [==============================] - 6s - loss: 0.8216 - val_loss: 1.0546


##  Neural net

In [50]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-5)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-5)

In [51]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1)(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [52]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=5, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 80158 samples, validate on 19846 samples
Epoch 1/5
80158/80158 [==============================] - 7s - loss: 1.7491 - val_loss: 0.8645
Epoch 2/5
80158/80158 [==============================] - 7s - loss: 0.9860 - val_loss: 0.8178
Epoch 3/5
80158/80158 [==============================] - 7s - loss: 0.9149 - val_loss: 0.8121
Epoch 4/5
80158/80158 [==============================] - 7s - loss: 0.8612 - val_loss: 0.8151
Epoch 5/5
80158/80158 [==============================] - 7s - loss: 0.8131 - val_loss: 0.8099
